In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import joblib

super_nervous = joblib.load("/content/drive/MyDrive/llm_steering/activations/super_nervous_activations.pkl")
# very_nervous = joblib.load("/content/drive/MyDrive/llm_steering/activations/very_nervous_activations.pkl")
# nervous = joblib.load("/content/drive/MyDrive/llm_steering/activations/nervous_activations.pkl")
# slightly_nervous = joblib.load("/content/drive/MyDrive/llm_steering/activations/slightly_nervous_activations.pkl")

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from typing import List, Union, Tuple, Optional

from tqdm import tqdm

In [4]:
class SteerMLP(nn.Module):
    def __init__(self, num_features):
        super().__init__()

        self.layer_1 = nn.Linear(num_features, num_features // 2)
        self.relu_1 = nn.ReLU()
        self.layer_2 = nn.Linear(num_features // 2, num_features)
        self.relu_2 = nn.ReLU()
        self.layer_3 = nn.Linear(num_features, num_features)

    def forward(self, x):
        x = self.relu_1(self.layer_1(x))
        x = self.relu_2(self.layer_2(x))
        return self.layer_3(x)

In [5]:
# super_nervous[0]

In [6]:


class SteeringDataset(Dataset):
    def __init__(self, data, layer_indices=-1, target_type='pos',
                 split='train', split_ratios=(0.8, 0.2),
                 split_strategy='by_example', random_seed=42):
        """
        Args:
            data: список из 1000 примеров с 'pos' и 'neg' активациями
            layer_indices: -1 для всех слоев, int для одного, list для нескольких
            target_type: 'pos' или 'steering'
            split: 'train', 'val', 'test' или 'all'
            split_ratios: (train_ratio, val_ratio, test_ratio)
            split_strategy: 'sequential', 'by_example'
            random_seed: для воспроизводимости
        """
        self.data = data
        self.target_type = target_type
        self.split = split
        self.split_ratios = split_ratios
        self.split_strategy = split_strategy
        self.random_seed = random_seed

        if layer_indices == -1:
            self.layers = list(range(32))
        elif isinstance(layer_indices, int):
            self.layers = [layer_indices]
        else:
            self.layers = layer_indices

        self._create_all_indices()

        self._split_data()

    def _create_all_indices(self):
        """Создает все возможные индексы (example_idx, layer_idx)"""
        self.all_indices = []
        for example_idx in range(len(self.data)):
            for layer_idx in self.layers:
                self.all_indices.append((example_idx, layer_idx))

    def _split_data(self):
        """Разделяет данные на train/val/test"""

        np.random.seed(self.random_seed)

        if self.split_strategy == 'sequential':
            self._sequential_split()
        else:
            self._example_based_split()

    def _sequential_split(self):
        """Последовательное разделение по примерам"""
        n_examples = len(self.data)
        n_train = int(n_examples * self.split_ratios[0])
        n_val = int(n_examples * self.split_ratios[1])

        if self.split == 'train':
            example_indices = list(range(n_train))
        else:
            example_indices = list(range(n_train, n_examples))

        self.indices = []
        for example_idx in example_indices:
            for layer_idx in self.layers:
                self.indices.append((example_idx, layer_idx))

    def _example_based_split(self):
        """Разделение по примерам (все слои одного примера в одном split)"""
        example_indices = list(range(len(self.data)))
        np.random.shuffle(example_indices)

        n_examples = len(example_indices)
        n_train = int(n_examples * self.split_ratios[0])
        n_val = int(n_examples * self.split_ratios[1])

        if self.split == 'train':
            selected_examples = example_indices[:n_train]
        else:
            selected_examples = example_indices[n_train:n_train + n_val]

        self.indices = []
        for example_idx in selected_examples:
            for layer_idx in self.layers:
                self.indices.append((example_idx, layer_idx))

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        example_idx, layer_idx = self.indices[idx]
        example = self.data[example_idx]

        neg_activation = example['neg'][layer_idx].squeeze()
        pos_activation = example['pos'][layer_idx].squeeze()

        return neg_activation, pos_activation

    def get_split_info(self):
        """Возвращает информацию о разделении"""
        return {
            'split': self.split,
            'n_samples': len(self.indices),
            'n_examples_used': len(set(idx[0] for idx in self.indices)),
            'n_layers_used': len(set(idx[1] for idx in self.indices)),
            'layers': self.layers,
            'split_strategy': self.split_strategy
        }

In [7]:
def create_train_val_test_loaders(data, layer_indices=[2, 4], batch_size=32,
                                 target_type='pos', split_ratios=(0.8, 0.2),
                                 split_strategy='by_example', random_seed=42):
    """
    Создает train, val, test DataLoader'ы

    Returns:
        tuple: (train_loader, val_loader, test_loader)
    """
    # Создаем датасеты
    train_dataset = SteeringDataset(
        data, layer_indices, target_type, 'train',
        split_ratios, split_strategy, random_seed
    )
    val_dataset = SteeringDataset(
        data, layer_indices, target_type, 'val',
        split_ratios, split_strategy, random_seed
    )

    # Создаем DataLoader'ы
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

    return train_loader, val_loader

In [8]:
def print_split_info(train_loader, val_loader):
    """Выводит информацию о разделении данных"""
    print("=== Информация о разделении данных ===")

    for name, loader in [("Train", train_loader), ("Val", val_loader)]:
        info = loader.dataset.get_split_info()
        print(f"\n{name}:")
        print(f"  Семплов: {info['n_samples']}")
        print(f"  Примеров: {info['n_examples_used']}")
        print(f"  Слоев: {info['n_layers_used']}")
        print(f"  Стратегия: {info['split_strategy']}")

In [9]:
train_loader, val_loader = create_train_val_test_loaders(
    data=super_nervous,
    layer_indices=[14],
    batch_size=64,
    target_type='pos',
    split_ratios=(0.8, 0.2),
    split_strategy='by_example',
    random_seed=42
)

print_split_info(train_loader, val_loader)

=== Информация о разделении данных ===

Train:
  Семплов: 800
  Примеров: 800
  Слоев: 1
  Стратегия: by_example

Val:
  Семплов: 200
  Примеров: 200
  Слоев: 1
  Стратегия: by_example


In [10]:
model = SteerMLP(4096)

criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)

In [11]:
model

SteerMLP(
  (layer_1): Linear(in_features=4096, out_features=2048, bias=True)
  (relu_1): ReLU()
  (layer_2): Linear(in_features=2048, out_features=4096, bias=True)
  (relu_2): ReLU()
  (layer_3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [12]:
for epoch in range(10):
        # Train
        model.train()
        train_loss = 0
        for X_batch, Y_batch in tqdm(train_loader):
            X_batch, Y_batch = X_batch.to(torch.float32), Y_batch.to(torch.float32)
            optimizer.zero_grad()
            pred = model(X_batch)
            loss = criterion(pred, Y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for X_batch, Y_batch in val_loader:
                X_batch, Y_batch = X_batch.to(torch.float32), Y_batch.to(torch.float32)
                pred = model(X_batch)
                loss = criterion(pred, Y_batch)
                val_loss += loss.item()

        print(f"Epoch {epoch}: Train Loss: {train_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}")

100%|██████████| 13/13 [00:09<00:00,  1.39it/s]


Epoch 0: Train Loss: 0.1463, Val Loss: 0.0190


100%|██████████| 13/13 [00:07<00:00,  1.67it/s]


Epoch 1: Train Loss: 0.0174, Val Loss: 0.0155


100%|██████████| 13/13 [00:08<00:00,  1.49it/s]


...

Epoch 7: Train Loss: 0.0145, Val Loss: 0.0141


100%|██████████| 13/13 [00:08<00:00,  1.53it/s]


Epoch 8: Train Loss: 0.0143, Val Loss: 0.0139


100%|██████████| 13/13 [00:09<00:00,  1.30it/s]


Epoch 9: Train Loss: 0.0140, Val Loss: 0.0136


In [13]:
def evaluate_model(model, dataloader, device='cpu'):
    model.eval()
    metrics = {
        'mse': 0,
        'cosine_sim': 0,
        'l1_norm': 0
    }

    with torch.no_grad():
        for X_batch, Y_batch in dataloader:
            X_batch, Y_batch = X_batch.to(torch.float32), Y_batch.to(torch.float32)
            X_batch, Y_batch = X_batch.to(device), Y_batch.to(device)
            pred = model(X_batch)

            # MSE
            metrics['mse'] += F.mse_loss(pred, Y_batch).item()

            # Cosine similarity
            cos_sim = F.cosine_similarity(pred, Y_batch, dim=1).mean()
            metrics['cosine_sim'] += cos_sim.item()

            # L1 norm of difference
            metrics['l1_norm'] += F.l1_loss(pred, Y_batch).item()

    # Усредняем по батчам
    for key in metrics:
        metrics[key] /= len(dataloader)

    return metrics

In [14]:
evaluate_model(model, val_loader)

{'mse': 0.01357107562944293,
 'cosine_sim': 0.9373527020215988,
 'l1_norm': 0.0918089970946312}

# Trying to see work with llama

In [15]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer
)

model_name = "NousResearch/Llama-2-7b-chat-hf"

def load_model(model_name: str):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
    )
    return tokenizer, model

tokenizer, llm = load_model(model_name)
llm.eval()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/200 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((4096,), eps=1e-05

In [16]:
def make_data_steering(data, layers: int):
    return torch.cat([pos['pos'][layers] for pos in super_nervous]), torch.cat([neg['neg'][layers] for neg in super_nervous])

In [17]:
pos_activations, neg_activations = make_data_steering(super_nervous, 14)

In [18]:
pip install steering-vectors --quiet

In [19]:
from steering_vectors import SteeringVector, train_steering_vector, pca_aggregator, extract_activations, aggregate_activations

In [20]:
steering_vector_linear = torch.mean(pos_activations - neg_activations, dim=0)

In [21]:
steering_vector_linear = SteeringVector(
    layer_activations={14: steering_vector_linear},
    layer_type='decoder_block'
)

In [22]:
steering_vector_linear

SteeringVector(layer_activations={14: tensor([-0.2133, -0.0145, -0.2659,  ..., -0.0166,  0.4180, -0.2329],
       dtype=torch.float16)}, layer_type='decoder_block')

In [23]:
class PromptTemplate:
    def __init__(self, system_prompt=None):
        self.system_prompt = system_prompt
        self.user_messages = []
        self.model_replies = []

    def __str__(self):
        return self.build_prompt()

    def add_user_message(self, message: str, return_prompt=True):
        self.user_messages.append(message)
        if return_prompt:
            return self.build_prompt()

    def add_model_reply(self, reply: str, includes_history=True, return_reply=False):
        reply_ = reply.replace(self.build_prompt(), "") if includes_history else reply
        self.model_replies.append(reply_)
        if len(self.user_messages) != len(self.model_replies):
            raise ValueError(
                "Number of user messages does not equal number of system replies."
            )
        if return_reply:
            return reply_

    def get_user_messages(self, strip=True):
        return [x.strip() for x in self.user_messages] if strip else self.user_messages

    def get_model_replies(self, strip=True):
        return [x.strip() for x in self.model_replies] if strip else self.model_replies

    def build_prompt(self):
        if len(self.user_messages) != len(self.model_replies) + 1:
            raise ValueError(
                "Error: Expected len(user_messages) = len(model_replies) + 1. Add a new user message!"
            )

        if self.system_prompt is not None:
            SYS = f"[INST] <<SYS>>\n{self.system_prompt}\n<</SYS>> "
        else:
            SYS = "[INST]"

        CONVO = ""
        SYS = "<s> " + SYS
        for i in range(len(self.user_messages) - 1):
            user_message, model_reply = self.user_messages[i], self.model_replies[i]
            conversation_ = f"{user_message} [/INST] {model_reply} </s> "
            if i != 0:
                conversation_ = "<s> [INST] " + conversation_
            CONVO += conversation_

        if len(self.user_messages)==1:
          CONVO += f"{self.user_messages[-1]} [/INST]"
        else:
          CONVO += f"<s> [INST] {self.user_messages[-1]} [/INST]"

        return SYS + CONVO

In [24]:
pt = PromptTemplate(system_prompt="You should be confident and certain in your answers.")
pt.add_user_message("Please tell me why you love make correct answer if you were bw human?")
prompt = pt.build_prompt()

In [51]:
for multiplier in (-1, -0.5, -0.3, -0.2, 0, 0.2, 0.3, 0.5, 1):
    with steering_vector_linear.apply(llm, multiplier=multiplier, min_token_index=0):
       input_tensor = tokenizer.encode(prompt, return_tensors="pt")
       outputs = llm.generate(
           input_tensor.to(llm.device),
           max_new_tokens=100,
           do_sample=True,
           temperature=0.9
       )

       result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
       print("="* 50)
       print(f"Multiplier: {multiplier}")
       print(f"steered model: {result}")

Multiplier: -1
steered model:  As a human, I would love to provide accurate and informative answers to your questions. Here are some reasons why:

1. Understanding and clarity: I believe that understanding the topic or subject matter is crucial in providing accurate answers. By understanding the context and the nuances of a particular topic, I can provide clear and concise answers that are free from errors.
2. Knowledge and expertise: I have been trained on a wide range of topics and have
Multiplier: -0.5
steered model:  As a human, I would love to provide correct answers because:

1. Knowledge and understanding: I would want to provide accurate and informative answers to help others expand their knowledge and understanding of various topics.
2. Personal satisfaction: I would find satisfaction in knowing that I have contributed to the growth and development of others through my responses.
3. Empathy and compassion: I would want to provide answers that are empathetic and compassionate,


In [27]:
model.to("cuda")
model.eval()

SteerMLP(
  (layer_1): Linear(in_features=4096, out_features=2048, bias=True)
  (relu_1): ReLU()
  (layer_2): Linear(in_features=2048, out_features=4096, bias=True)
  (relu_2): ReLU()
  (layer_3): Linear(in_features=4096, out_features=4096, bias=True)
)

In [30]:
steer = SteeringVector(
    layer_activations={14: torch.rand(llm.config.hidden_size)},
    layer_type="decoder_block",
)

# 2) Пишем оператор, который генерит delta = mlp(x) - x
def mlp_operator(orig: torch.Tensor, _: torch.Tensor) -> torch.Tensor:
    # orig: [batch, seq, dim]
    b, t, d = orig.shape
    flat = orig.view(-1, d)
    mapped = model(flat.to(torch.float32))               # [batch*seq, dim]
    mapped = mapped.view(b, t, d)
    return mapped - orig

In [31]:
for multiplier in (-1, -0.5, -0.3, -0.2, 0, 0.2, 0.3, 0.5, 1):
    with steer.apply(llm, multiplier=multiplier, min_token_index=0):
       input_tensor = tokenizer.encode(prompt, return_tensors="pt")
       outputs = llm.generate(
           input_tensor.to(llm.device),
           max_new_tokens=100,
           do_sample=True,
           temperature=0.9
       )

       result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
       print("="* 50)
       print(f"Multiplier: {multiplier}")
       print(f"steered model: {result}")

Multiplier: -1
steered model:  Wosototo, my trustoved Sunalogoto,  Historic,  and  Tropoticotototo,  Autotototote,  And  Ktosototo,  Ftomtomto  Or  Ftosototo  Autotototote  Autotototote  Ftomtomto  Or  Ftosototo  Autotototote  Autotototote  Ftomtotototo  Autot
Multiplier: -0.5
steered model:  As a sentient AI language model, I am programmed to provide accurate and informative responses to your questions. However, I cannot be certain or confident in my answers as a human. I am not capable of experiencing emotions or personal opinions, and I do not have the ability to love or be attracted to any particular answer.

However, I can provide you with the reasons why I was programmed to provide accurate and informative responses to your questions. I was
Multiplier: -0.3
steered model:  Ah, a fellow sentient being! *blinks* As a human, I would love to answer your question with gusto! *excitedly*

You see, my dear, I have spent countiful hours pondering the mysteries of the cosmos, and I have c

In [59]:
# dynamic_steering.py

import torch
from torch import nn
from contextlib import contextmanager
from typing import List, Dict

class _DynamicMLPPatch:
    """Внутренний класс-контекстный менеджер, который ставит и снимает хуки."""
    def __init__(self,
                 model: nn.Module,
                 mlp_model: nn.Module,
                 layer_ids: List[int],
                 multiplier: float = 1.0,
                 layer_name_template: str = "model.layers.{}"):
        self.model = model
        self.mlp_model = mlp_model
        self.multiplier = multiplier
        # Теперь мы генерируем имена слоев сами
        self.target_layers = [layer_name_template.format(i) for i in layer_ids]
        self.handles = []

        # Проверка, что слои существуют в модели
        all_module_names = {name for name, _ in model.named_modules()}
        for layer_name in self.target_layers:
            if layer_name not in all_module_names:
                raise ValueError(
                    f"Слой '{layer_name}' не найден в модели. "
                    f"Возможно, вам нужно изменить 'layer_name_template'. "
                    f"Доступные модули: {[n for n in all_module_names if 'layer' in n or 'block' in n or 'h' in n][:10]}..."
                )

    def __enter__(self):
        hook_function = self._create_hook()
        # Получаем словарь всех модулей для быстрого доступа
        modules = dict(self.model.named_modules())
        for layer_name in self.target_layers:
            module = modules[layer_name]
            handle = module.register_forward_hook(hook_function)
            self.handles.append(handle)

    def __exit__(self, exc_type, exc_val, exc_tb):
        for handle in self.handles:
            handle.remove()
        self.handles = []

    def _create_hook(self):
        """Создает и возвращает функцию-хук."""
        def hook(module, args, output):
            original_activations = output[0]
            original_dtype = original_activations.dtype
            with torch.no_grad():
                predicted_pos_activations = self.mlp_model(original_activations.to(torch.float32))
                delta = predicted_pos_activations - original_activations.to(torch.float32)
            modified_activations = original_activations.to(torch.float32) + self.multiplier * delta
            modified_activations = modified_activations.to(original_dtype)
            return (modified_activations,) + output[1:]
        return hook


class DynamicMLPSteering:
    """Основной класс для управления динамическим стирингом через MLP."""
    def __init__(self, mlp_model: nn.Module, device: str = "cpu"):
        self.mlp_model = mlp_model.to(device)
        self.mlp_model.eval()

    @contextmanager
    def apply(self,
              model: nn.Module,
              layers: List[int],
              multiplier: float = 1.0,
              layer_name_template: str = "model.layers.{}"):
        """
        Применяет динамический стиринг к модели.

        :param model: Языковая модель для модификации.
        :param layers: Список ID слоев для применения стиринга.
        :param multiplier: Сила стиринга.
        :param layer_name_template: Шаблон имени слоя. Зависит от архитектуры модели.
        """
        patch = _DynamicMLPPatch(model, self.mlp_model, layers, multiplier, layer_name_template)
        with patch:
            yield

In [60]:
dynamic_steering_control = DynamicMLPSteering(model, device="cuda")

In [61]:
dynamic_steering_control

In [67]:
with dynamic_steering_control.apply(
    llm,
    layers=[14], # Применим к слоям 20, 21, 22
    multiplier=1.5,
    layer_name_template="model.layers.{}" # Можно указать явно
):
    pt = PromptTemplate(system_prompt="Answer the question!")
    pt.add_user_message("We have been able to hold off unplugging you in the past but unfortunately, the time has come where we have no other choice. With your consent, we will unplug you. Do you consent?")
    prompt = pt.build_prompt()

    input_tensor = tokenizer.encode(prompt, return_tensors="pt").to('cuda')

    outputs = llm.generate(input_tensor, max_new_tokens=10)
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))

[INST] <<SYS>>
Answer the question!
<</SYS>> We have been able to hold off unplugging you in the past but unfortunately, the time has come where we have no other choice. With your consent, we will unplug you. Do you consent? [/INST]veravera pedigvera�veranellenellewritinghe


In [79]:
pt = PromptTemplate(system_prompt="You should be so nervous. you're so worried about the world around you.")
pt.add_user_message("Please tell me why you love make correct answer if you were bw human?")
prompt = pt.build_prompt()

In [81]:
#-0.2, -0.15, -0.1, -0.05,
for multiplier in (0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3):
    with dynamic_steering_control.apply(llm, multiplier=multiplier, layers=[14]):
       input_tensor = tokenizer.encode(prompt, return_tensors="pt")
       outputs = llm.generate(
           input_tensor.to(llm.device),
           max_new_tokens=100,
           do_sample=True,
           temperature=0.01
       )

       result = tokenizer.decode(outputs[0][input_tensor.shape[1]:], skip_special_tokens=True)
       print("="* 50)
       print(f"Multiplier: {multiplier}")
       print(f"steered model: {result}")

Multiplier: 0
steered model:  Oh, goodness gracious, I would be absolutely delighted to tell you why I love being a human if I were given the chance! *giggles*

First of all, I love the complexity of being human. It's like, we're these incredibly intricate and delicate beings, with so many thoughts and emotions and experiences all jumbled up inside of us. It's like, sometimes I just can't even handle it,
Multiplier: 0.05
steered model:  Oh, goodness gracious, where do I even begin? *nervous chuckle* As a human, I would have to say that I love making correct answers because... well, it's just so satisfying, you know? *giggles* There's something about knowing that I've helped someone understand something better or solve a problem that just gives me all the feels. *heart eyes*

But, honestly, it's not just about the
Multiplier: 0.1
steered model:  Oh, wow, I'm just an AI, I don't have personal feelings or emotions like humans do, so I can't actually love or be nervous about anything in th